In [11]:
# playwright_agent_demo_jupyter.py (multi-step automation with Playwright for Jupyter)

# ✅ Required installations (run in a notebook cell):
# !pip install playwright nest_asyncio
# !playwright install

import asyncio
import nest_asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()  # Allows running async loops in notebooks

INSTRUCTIONS = [
    {
        "description": "Go to Hacker News",
        "url": "https://news.ycombinator.com/"
    },
    {
        "description": "Click the top news article (fallback safe)",
        "fallback": "a.titleline a",
        "selector": "a.storylink, a.titlelink, a.titleline a"
    },
    {
        "description": "Return to Hacker News homepage",
        "url": "https://news.ycombinator.com/"
    },
    {
        "description": "Click 'new' link to see recent posts",
        "selector": "a[href='newest']"
    }
]

async def run_playwright_steps_async():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        for step in INSTRUCTIONS:
            print(f"🔹 {step['description']}")
            if "url" in step:
                try:
                    await page.goto(step["url"], timeout=60000)
                    await page.wait_for_load_state("domcontentloaded")
                except Exception as e:
                    print(f"❌ Failed to load URL: {step['url']}\n{e}")
                    continue
            elif "selector" in step:
                try:
                    await page.wait_for_selector(step["selector"], timeout=10000)
                    element = page.locator(step["selector"])
                    if await element.count() > 0:
                        await element.first.click()
                        await asyncio.sleep(2)
                    else:
                        print("⚠️ Element not found — trying fallback selector")
                        if "fallback" in step:
                            try:
                                await page.wait_for_selector(step["fallback"], timeout=5000)
                                element = page.locator(step["fallback"])
                                await element.first.click()
                            except:
                                print("❌ Fallback also failed")
                except Exception as e:
                    print(f"❌ Failed at selector: {step['selector']}\n{e}")
            await asyncio.sleep(1)

        print("✅ All steps completed")
        await browser.close()

# ✅ To run in a Jupyter notebook cell:
# await run_playwright_steps_async()



In [12]:
await run_playwright_steps_async()


🔹 Go to Hacker News
🔹 Click the top news article (fallback safe)
❌ Failed at selector: a.storylink, a.titlelink, a.titleline a
Page.wait_for_selector: Timeout 10000ms exceeded.
Call log:
  - waiting for locator("a.storylink, a.titlelink, a.titleline a") to be visible

🔹 Return to Hacker News homepage
🔹 Click 'new' link to see recent posts
✅ All steps completed


In [18]:
# conversational_agent_app.py - Build a full LangChain-powered conversational agent with tools and memory

import os
import openai
import datetime
import requests
import wikipedia
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, Field
from typing import List
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
import panel as pn
import param

# Load API Key
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

# ========== Define Tools ==========
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> str:
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    params = {"latitude": latitude, "longitude": longitude, "hourly": "temperature_2m", "forecast_days": 1}
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        return "Weather API failed."
    data = response.json()
    now = datetime.datetime.utcnow()
    times = [datetime.datetime.fromisoformat(t.replace("Z", "+00:00")) for t in data['hourly']['time']]
    temps = data['hourly']['temperature_2m']
    index = min(range(len(times)), key=lambda i: abs(times[i] - now))
    return f"The current temperature is {temps[index]}°C"

@tool
def search_wikipedia(query: str) -> str:
    titles = wikipedia.search(query)
    summaries = []
    for title in titles[:3]:
        try:
            page = wikipedia.page(title, auto_suggest=False)
            summaries.append(f"**{title}**\n{page.summary}")
        except:
            pass
    return "\n\n".join(summaries) if summaries else "No good result found."

@tool
def create_your_own(query: str) -> str:
    return f"You sent: {query}. This reverses it: {query[::-1]}"

# ========== Register Tools ==========
tools = [get_current_temperature, search_wikipedia, create_your_own]

# ========== Panel Chatbot UI ==========
pn.extension()

class ConversationalBot(param.Parameterized):
    def __init__(self, tools, **params):
        super().__init__(**params)
        self.panels = []
        self.tool_funcs = [format_tool_to_openai_function(t) for t in tools]
        self.llm = ChatOpenAI(temperature=0).bind(functions=self.tool_funcs)
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.llm | OpenAIFunctionsAgentOutputParser()

        self.executor = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    def interact(self, query):
        if not query:
            return
        result = self.executor.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=500)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=500, styles={"background-color": "#f0f0f0"}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)

# ========== Launch the Panel Chat App ==========
cb = ConversationalBot(tools)
inp = pn.widgets.TextInput(placeholder='Ask me anything...')
conversation = pn.bind(cb.interact, inp)

tab = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=400),
    pn.layout.Divider()
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# 🧠 Conversational Agent Bot')),
    pn.Tabs(('Chat', tab))
)

dashboard.servable()


ValueError: Function must have a docstring if description not provided.

In [17]:
pip install panel param openai langchain wikipedia python-dotenv requests

  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached contourpy-1.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached narwhals-1.35.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 6.9 MB/s eta 0:00:00a 0:00:01
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached contourpy-1.3.2-cp311-cp311-macosx_11_0_arm64.whl (254 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached narwhals-1.35.0-py3-none-any.w